## In this notebook, we will be inspecting data from cancelled subscribers, tidying this data, testing it for automation and later showing our findings.

### First, we will carry out the necessary imports and change the working directory.

In [1]:
import sqlite3    #import SQLite library, which allows interation with SQLite databases
import pandas as pd    #import pandas - a library used for data manipulation
import ast    #imports ast - used for identifying errors within the code  # CHECK THIS HAS BEEN USED
import json
import numpy as np    #import NumPy - used to add useful mathematical functions
import os    #imports os - used to interact with the importating systenm
from datetime import datetime    #imports datetime module allowing better manipulation of dates and times

In [2]:
os.chdir(r'\Users\maxpe\OneDrive\Documents\Codecademy\Cancelled Subscribers Automated Data Ingestion\subscriber-pipeline-starter-kit\dev') #changes the working directory - the 'r' is used to tell Python to treat the backslashes as characters, rather than an escape character
pd.set_option('display.max_colwidth', None) # pandas may cut out information on a dataframe if there are too many characters, this will override that.

### Below, we will establish a connection to the database, and then run a query to find what tables we will be working with.

In [3]:
con = sqlite3.connect('cademycode.db') #this connects to cademycode.db
cur = con.cursor() #this creates a cursor object - which will allow us to manipulate the data with Python
#the below query creates an empty list, as the loop iterates through the database, each time it finds a table, it will add it to the list, and then print it out, allowing us to see what tables are in the database
table_list = []
for row in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'"):
    table_list.append(row[0])
print(table_list)

['cademycode_students', 'cademycode_courses', 'cademycode_student_jobs', 'cademycode_cleaned', 'cademycode_cleaned.db']


### Now we know the tables, we can save these as Python variables to enable maniuplation

In [4]:
# the next 3 queries takes all the rows from the tables in the table_list output, and loads them into dataframes using pandas.
students = pd.read_sql_query("SELECT * FROM cademycode_students", con)
courses = pd.read_sql_query("SELECT * FROM cademycode_courses", con)
student_jobs = pd.read_sql_query("SELECT * FROM cademycode_student_jobs", con)

# these print statement show the number of rows of data within each dataframe
print('students:', len(students))
print('courses:', len(courses))
print('student_jobs:', len(student_jobs))

students: 5000
courses: 10
student_jobs: 13


### We now have each table as a dataframe, it is time to inspect these tables, and look for data anomolies.

In [5]:
# We will start with the students table, first, we will call .head(), this will show the first 5 rows of the data, allowing us to get a sense of the table.
students.head()

,uuid,name,dob,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs
0,1,Annabelle Avery,1943-07-03,F,"{""mailing_address"": ""303 N Timber Key, Irondale, Wisconsin, 84736"", ""email"": ""annabelle_avery9376@woohoo.com""}",7.0,6.0,1.0,4.99
1,2,Micah Rubio,1991-02-07,M,"{""mailing_address"": ""767 Crescent Fair, Shoals, Indiana, 37439"", ""email"": ""rubio6772@hmail.com""}",7.0,5.0,8.0,4.4
2,3,Hosea Dale,1989-12-07,M,"{""mailing_address"": ""P.O. Box 41269, St. Bonaventure, Virginia, 83637"", ""email"": ""hosea_dale8084@coldmail.com""}",7.0,8.0,8.0,6.74
3,4,Mariann Kirk,1988-07-31,F,"{""mailing_address"": ""517 SE Wintergreen Isle, Lane, Arkansas, 82242"", ""email"": ""kirk4005@hmail.com""}",6.0,7.0,9.0,12.31
4,5,Lucio Alexander,1963-08-31,M,"{""mailing_address"": ""18 Cinder Cliff, Doyles borough, Rhode Island, 73737"", ""email"": ""alexander9810@hmail.com""}",7.0,14.0,3.0,5.64


In [6]:
students.info()
# calling .info() will give us details of the column names, the count and the datatype.
# we can see tables 5,6,7,8 have some missing values.
# we see some datatypes arent correct, dob should be date time, time_spent_hrs, current_career_path_id, num_course_taken and job_id should also be updated to floats / integers
# contact_info also needs breaking up into separate columns for each part of the address and email address.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uuid                    5000 non-null   int64 
 1   name                    5000 non-null   object
 2   dob                     5000 non-null   object
 3   sex                     5000 non-null   object
 4   contact_info            5000 non-null   object
 5   job_id                  4995 non-null   object
 6   num_course_taken        4749 non-null   object
 7   current_career_path_id  4529 non-null   object
 8   time_spent_hrs          4529 non-null   object
dtypes: int64(1), object(8)
memory usage: 351.7+ KB


In [7]:
# this selects the 'dob' column from students, and converts this to a data time format with pandas
students['dob'] = pd.to_datetime(students['dob'])  # Convert dob to datetime

# This creates an 'age' column, and assigns the students age by acquiring the current date.
# After current date is acquired, it finds the days passed since the students dob and divides this by 365, to find the year.
students['age'] = (datetime.now() - students['dob']).dt.days // 365

# call .head() to confirm the new age column
students.head()


,uuid,name,dob,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs,age
0,1,Annabelle Avery,1943-07-03,F,"{""mailing_address"": ""303 N Timber Key, Irondale, Wisconsin, 84736"", ""email"": ""annabelle_avery9376@woohoo.com""}",7.0,6.0,1.0,4.99,81
1,2,Micah Rubio,1991-02-07,M,"{""mailing_address"": ""767 Crescent Fair, Shoals, Indiana, 37439"", ""email"": ""rubio6772@hmail.com""}",7.0,5.0,8.0,4.4,33
2,3,Hosea Dale,1989-12-07,M,"{""mailing_address"": ""P.O. Box 41269, St. Bonaventure, Virginia, 83637"", ""email"": ""hosea_dale8084@coldmail.com""}",7.0,8.0,8.0,6.74,34
3,4,Mariann Kirk,1988-07-31,F,"{""mailing_address"": ""517 SE Wintergreen Isle, Lane, Arkansas, 82242"", ""email"": ""kirk4005@hmail.com""}",6.0,7.0,9.0,12.31,36
4,5,Lucio Alexander,1963-08-31,M,"{""mailing_address"": ""18 Cinder Cliff, Doyles borough, Rhode Island, 73737"", ""email"": ""alexander9810@hmail.com""}",7.0,14.0,3.0,5.64,61


In [8]:
# We will remove contact_info, and replace it with columns 'street' 'city' 'state' 'post_code' and 'email'
# The data is stored as JSON, so we use the json module and convert the JSON into a dictionary, allowing us to work with it

def parse_contact_info(info):
    # This converts the JSON string to a Python dictionary
    parsed_info = json.loads(info)  
    mailing_address = parsed_info['mailing_address']
    
    # This will split the key 'mailing_address' into 4 variables, using the comma as a separator
    address_parts = mailing_address.split(', ')
    if len(address_parts) == 4:
        street = address_parts[0]
        city = address_parts[1]
        state = address_parts[2]
        post_code = address_parts[3]

    # The email key does not need separating so it is easier to work with
    email = parsed_info['email']
    
    return pd.Series([street, city, state, post_code, email])

students[['street', 'city', 'state', 'post_code', 'email']] = students['contact_info'].apply(parse_contact_info)

# finally, we remove the contact_info column
students.drop(columns=['contact_info'], inplace=True)


In [9]:
# checking the new fields have been added
students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    5000 non-null   int64         
 1   name                    5000 non-null   object        
 2   dob                     5000 non-null   datetime64[ns]
 3   sex                     5000 non-null   object        
 4   job_id                  4995 non-null   object        
 5   num_course_taken        4749 non-null   object        
 6   current_career_path_id  4529 non-null   object        
 7   time_spent_hrs          4529 non-null   object        
 8   age                     5000 non-null   int64         
 9   street                  5000 non-null   object        
 10  city                    5000 non-null   object        
 11  state                   5000 non-null   object        
 12  post_code               5000 non-null   object  

In [10]:
# Now, we change the data types of certain columns to floats
students['job_id'] = students['job_id'].astype(float)
students['num_course_taken'] = students['num_course_taken'].astype(float)
students['current_career_path_id'] = students['current_career_path_id'].astype(float)
students['time_spent_hrs'] = students['time_spent_hrs'].astype(float)

In [11]:
# Checking data types have updated
students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    5000 non-null   int64         
 1   name                    5000 non-null   object        
 2   dob                     5000 non-null   datetime64[ns]
 3   sex                     5000 non-null   object        
 4   job_id                  4995 non-null   float64       
 5   num_course_taken        4749 non-null   float64       
 6   current_career_path_id  4529 non-null   float64       
 7   time_spent_hrs          4529 non-null   float64       
 8   age                     5000 non-null   int64         
 9   street                  5000 non-null   object        
 10  city                    5000 non-null   object        
 11  state                   5000 non-null   object        
 12  post_code               5000 non-null   object  

In [12]:
# Checking for duplicates; all fields return false, indicating no duplicated data
students.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
4995    False
4996    False
4997    False
4998    False
4999    False
Length: 5000, dtype: bool

In [13]:
# We will now look into the missing values in job_id
missing_job_id = students[students['job_id'].isnull()]
missing_job_id

,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,age,street,city,state,post_code,email
162,163,Glen Riley,2002-08-22,M,NaN,8.0,3.0,5.70,22,P.O. Box 37267,Cornlea village,Tennessee,19192,glen_riley4484@hmail.com
757,758,Mercedez Vorberg,2002-03-25,F,NaN,15.0,4.0,4.14,22,284 Cedar Seventh,Virden village,Washington,60489,mercedez6297@woohoo.com
854,855,Kurt Ho,2002-05-29,M,NaN,0.0,8.0,23.72,22,P.O. Box 27254,Olin,New Hampshire,60067,ho6107@inlook.com
1029,1030,Penny Gaines,2002-03-01,N,NaN,15.0,4.0,16.25,22,138 Misty Vale,Stockton borough,West Virginia,53630,gaines2897@hmail.com
1542,1543,Frederick Reilly,2002-11-13,M,NaN,7.0,9.0,21.32,21,P.O. Box 40769,Quakervillage,Maryland,96218,frederick_reilly6971@woohoo.com


In [14]:
# There are no obvious correlations as to why these are missing, due to only 5 rows from 1,000 being missing,
# We will delete the missing values, as this should not impact our dataset.
students = students.dropna(subset=['job_id'])
students.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4995 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    4995 non-null   int64         
 1   name                    4995 non-null   object        
 2   dob                     4995 non-null   datetime64[ns]
 3   sex                     4995 non-null   object        
 4   job_id                  4995 non-null   float64       
 5   num_course_taken        4744 non-null   float64       
 6   current_career_path_id  4524 non-null   float64       
 7   time_spent_hrs          4524 non-null   float64       
 8   age                     4995 non-null   int64         
 9   street                  4995 non-null   object        
 10  city                    4995 non-null   object        
 11  state                   4995 non-null   object        
 12  post_code               4995 non-null   object       

In [15]:
# This will show the first 50 records where num_course_taken is null
missing_num_course_taken = students[students['num_course_taken'].isnull()]
missing_num_course_taken.head(50)

,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,age,street,city,state,post_code,email
25,26,Doug Browning,1970-06-08,M,7.0,NaN,5.0,1.92,54,P.O. Box 15845,Devine,Florida,23097,doug7761@inlook.com
26,27,Damon Schrauwen,1953-10-31,M,4.0,NaN,10.0,3.73,71,P.O. Box 84659,Maben,Georgia,66137,damon9864@woohoo.com
51,52,Alisa Neil,1977-05-28,F,5.0,NaN,8.0,22.86,47,16 View Annex,Mosses,North Dakota,25748,alisa9616@inlook.com
70,71,Chauncey Hooper,1962-04-07,M,3.0,NaN,3.0,3.97,62,955 Dewy Flat,Slaughterville,South Carolina,22167,chauncey6352@woohoo.com
80,81,Ellyn van Heest,1984-06-28,F,3.0,NaN,10.0,12.39,40,872 Cider Glade,Chicken,Delaware,42689,ellyn_vanheest8375@hmail.com
96,97,Moises Krekel,2004-06-15,M,8.0,NaN,NaN,NaN,20,455 Silent Smith,Radium Springs,North Carolina,29278,moises_krekel5984@woohoo.com
139,140,Charlie Semvoort,2004-03-06,N,8.0,NaN,5.0,5.76,20,76 Dewy Quail,Whitesburg,South Carolina,24297,charlie8186@inlook.com
149,150,Lindsey Mcclain,1972-11-30,N,6.0,NaN,9.0,21.79,51,751 Robin Highlands,Hurlock,Tennessee,33406,lindsey_mcclain8692@woohoo.com
157,158,Ching Rijkensz,1988-02-17,N,3.0,NaN,7.0,2.36,36,471 Timber Arbor,Oaks,Texas,84192,ching_rijkensz6414@hmail.com
202,203,Bettie Torres,1973-06-05,N,6.0,NaN,9.0,6.04,51,457 SW Valley,New Madison village,South Carolina,78280,bettie_torres6037@hmail.com


In [16]:
# After investigating the missing data for num_course_taken, we can not see any correlations with the other columns as to why this is
# Due to us having no access to a data team, or data entry team, we cannot investigate how the data is being added
# Due to the data being approximately 5% of the set, we will remove this to allow for more accurate analysis on the remaining set
students = students.dropna(subset=['num_course_taken'])
students.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4744 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    4744 non-null   int64         
 1   name                    4744 non-null   object        
 2   dob                     4744 non-null   datetime64[ns]
 3   sex                     4744 non-null   object        
 4   job_id                  4744 non-null   float64       
 5   num_course_taken        4744 non-null   float64       
 6   current_career_path_id  4293 non-null   float64       
 7   time_spent_hrs          4293 non-null   float64       
 8   age                     4744 non-null   int64         
 9   street                  4744 non-null   object        
 10  city                    4744 non-null   object        
 11  state                   4744 non-null   object        
 12  post_code               4744 non-null   object       

In [17]:
# This will show the first 50 records where current_career_path_id is null
missing_current_career_path_id = students[students['current_career_path_id'].isnull()]
missing_current_career_path_id.head(50)

,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,age,street,city,state,post_code,email
15,16,Norene Dalton,1976-04-30,F,6.0,0.0,NaN,NaN,48,130 Wishing Essex,Branch,Ohio,13616,norene_dalton9509@hmail.com
19,20,Sofia van Steenbergen,1990-02-21,N,7.0,13.0,NaN,NaN,34,634 Clear Barn Dell,Beaman,Georgia,33288,vansteenbergen8482@inlook.com
30,31,Christoper Warner,1989-12-28,M,2.0,5.0,NaN,NaN,34,556 Stony Highlands,Drain,Illinois,01973,warner5906@coldmail.com
49,50,Antony Horne,1996-05-29,M,3.0,2.0,NaN,NaN,28,P.O. Box 78685,Lenox,Texas,15516,antony577@coldmail.com
54,55,Omar Bunk,1955-11-08,M,3.0,14.0,NaN,NaN,69,445 Dale Hollow,Vermont village,South Carolina,28329,omar1245@coldmail.com
55,56,Lincoln Hartman,1950-07-15,M,5.0,8.0,NaN,NaN,74,P.O. Box 22581,Southmont borough,West Virginia,03735,hartman6763@hmail.com
62,63,Zana Willems,1953-11-23,F,2.0,2.0,NaN,NaN,70,319 Iron Treasure,Thermalito,Virginia,15241,willems4043@woohoo.com
64,65,Jasmine Vos,1942-08-24,F,5.0,13.0,NaN,NaN,82,P.O. Box 56722,La Veta,Vermont,72685,vos8677@coldmail.com
71,72,Genevie Huerta,1954-11-15,F,1.0,8.0,NaN,NaN,70,401 Quaking Smith,McDermitt,Montana,96073,genevie_huerta5604@inlook.com
88,89,Scott de Boef,1976-01-21,M,2.0,11.0,NaN,NaN,48,P.O. Box 47591,Pemberton borough,Pennsylvania,13428,scott_deboef9061@coldmail.com


In [18]:
# Looking at the data above, it is obvious to see that when a student is not enrolled on a current career path, there are no respective hours spent.
# To maintain the dataset, we will fill the NaN's with a 0 for both columns as we have already deleted approximately 5% of data.
students['current_career_path_id'].fillna(0, inplace=True)
students['time_spent_hrs'].fillna(0, inplace=True)
students.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4744 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    4744 non-null   int64         
 1   name                    4744 non-null   object        
 2   dob                     4744 non-null   datetime64[ns]
 3   sex                     4744 non-null   object        
 4   job_id                  4744 non-null   float64       
 5   num_course_taken        4744 non-null   float64       
 6   current_career_path_id  4744 non-null   float64       
 7   time_spent_hrs          4744 non-null   float64       
 8   age                     4744 non-null   int64         
 9   street                  4744 non-null   object        
 10  city                    4744 non-null   object        
 11  state                   4744 non-null   object        
 12  post_code               4744 non-null   object       

In [19]:
# And now, the courses dataframe:
# Due to the students who werent on career path, which we replaced NaN's with 0's, we will create a career path of 'Not Applicable'
# This placeholder will help the data integrity during the merge
not_applicable = {
    'career_path_id': 0,
    'career_path_name': 'Not Applicable',
    'hours_to_complete': 0
}
not_applicable_df = pd.DataFrame([not_applicable])  # This converts the above dictionary into a dataframe which allows pd.concat to merge courses with not_applicable
courses = pd.concat([courses, not_applicable_df], ignore_index=True)
courses.head(50)


,career_path_id,career_path_name,hours_to_complete
0,1,data scientist,20
1,2,data engineer,20
2,3,data analyst,12
3,4,software engineering,25
4,5,backend engineer,18
5,6,frontend engineer,20
6,7,iOS developer,27
7,8,android developer,27
8,9,machine learning engineer,35
9,10,ux/ui designer,15


In [20]:
courses.info()
# Again, everything looks as it should below.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   career_path_id     11 non-null     int64 
 1   career_path_name   11 non-null     object
 2   hours_to_complete  11 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 396.0+ bytes


In [21]:
# Finally, the student_jobs dataframe:
student_jobs.head(50)

,job_id,job_category,avg_salary
0,1,analytics,86000
1,2,engineer,101000
2,3,software developer,110000
3,4,creative,66000
4,5,financial services,135000
5,6,education,61000
6,7,HR,80000
7,8,student,10000
8,9,healthcare,120000
9,0,other,80000


In [22]:
student_jobs = student_jobs.drop_duplicates()
# There are a couple of duplicates which we have cleared up

In [23]:
student_jobs.info()
# Below, these look fine, which cleans up the 3 dataframes!

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   job_id        10 non-null     int64 
 1   job_category  10 non-null     object
 2   avg_salary    10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 320.0+ bytes


In [24]:
students.info()
courses.info()
student_jobs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4744 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    4744 non-null   int64         
 1   name                    4744 non-null   object        
 2   dob                     4744 non-null   datetime64[ns]
 3   sex                     4744 non-null   object        
 4   job_id                  4744 non-null   float64       
 5   num_course_taken        4744 non-null   float64       
 6   current_career_path_id  4744 non-null   float64       
 7   time_spent_hrs          4744 non-null   float64       
 8   age                     4744 non-null   int64         
 9   street                  4744 non-null   object        
 10  city                    4744 non-null   object        
 11  state                   4744 non-null   object        
 12  post_code               4744 non-null   object       

### We now have the dataframes cleaned, we will now merge them into one, export into a database and then create a CSV

In [25]:
# First, we will merge students with student_jobs
# We will join all tables on job_id, as this field is present in each table
# We will use left joins to keep flexibility and ensure all records are kept
# On the final merge, we use left_on and right_on to link the student_jobs_merged and courses on current_career_path_id and career_path_id
final_df = students.merge(courses, left_on='current_career_path_id', right_on='career_path_id', how='left')
final_df = final_df.merge(student_jobs, on='job_id', how='left')
final_df.head(50)




,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,age,street,city,state,post_code,email,career_path_id,career_path_name,hours_to_complete,job_category,avg_salary
0,1,Annabelle Avery,1943-07-03,F,7.0,6.0,1.0,4.99,81,303 N Timber Key,Irondale,Wisconsin,84736,annabelle_avery9376@woohoo.com,1,data scientist,20,HR,80000
1,2,Micah Rubio,1991-02-07,M,7.0,5.0,8.0,4.40,33,767 Crescent Fair,Shoals,Indiana,37439,rubio6772@hmail.com,8,android developer,27,HR,80000
2,3,Hosea Dale,1989-12-07,M,7.0,8.0,8.0,6.74,34,P.O. Box 41269,St. Bonaventure,Virginia,83637,hosea_dale8084@coldmail.com,8,android developer,27,HR,80000
3,4,Mariann Kirk,1988-07-31,F,6.0,7.0,9.0,12.31,36,517 SE Wintergreen Isle,Lane,Arkansas,82242,kirk4005@hmail.com,9,machine learning engineer,35,education,61000
4,5,Lucio Alexander,1963-08-31,M,7.0,14.0,3.0,5.64,61,18 Cinder Cliff,Doyles borough,Rhode Island,73737,alexander9810@hmail.com,3,data analyst,12,HR,80000
5,6,Shavonda Mcmahon,1989-10-15,F,6.0,10.0,3.0,10.12,35,P.O. Box 81591,Tarpon Springs,Montana,37057,shavonda5863@coldmail.com,3,data analyst,12,education,61000
6,7,Terrell Bleijenberg,1959-05-05,M,2.0,9.0,8.0,24.17,65,P.O. Box 53471,Oskaloosa,Virginia,85274,bleijenberg188@hmail.com,8,android developer,27,engineer,101000
7,8,Stanford Allan,1997-11-22,M,3.0,3.0,1.0,19.54,26,255 Spring Avenue,Point Baker,Texas,15796,stanford_allan8055@coldmail.com,1,data scientist,20,software developer,110000
8,9,Tricia Delacruz,1961-10-20,F,1.0,6.0,9.0,1.75,63,997 Dewy Apple,Lake Lindsey,Washington,78266,tricia_delacruz6622@woohoo.com,9,machine learning engineer,35,analytics,86000
9,10,Regenia van der Helm,1999-02-23,N,5.0,7.0,6.0,13.55,25,220 Middle Ridge,Falcon Heights,New Mexico,46971,regenia6908@inlook.com,6,frontend engineer,20,financial services,135000


In [26]:
final_df.info()
# As we can see, the final dataframe has the same non-null count as before!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4744 entries, 0 to 4743
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    4744 non-null   int64         
 1   name                    4744 non-null   object        
 2   dob                     4744 non-null   datetime64[ns]
 3   sex                     4744 non-null   object        
 4   job_id                  4744 non-null   float64       
 5   num_course_taken        4744 non-null   float64       
 6   current_career_path_id  4744 non-null   float64       
 7   time_spent_hrs          4744 non-null   float64       
 8   age                     4744 non-null   int64         
 9   street                  4744 non-null   object        
 10  city                    4744 non-null   object        
 11  state                   4744 non-null   object        
 12  post_code               4744 non-null   object  

In [27]:
# Now to create the cleansed database and CSV!
final_df.to_sql('cademycode_cleaned.db', con, if_exists='replace', index=False)
final_df.to_csv('cademycode_cleaned.csv', index=False)